In [ ]:
import yfinance as yf
from isyatirimhisse import StockData
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
bist100 = yf.download('XU100.IS', start='2019-01-01')
bist100['Date'] = pd.to_datetime(bist100.index)

plt.figure(figsize=(12,6))
plt.plot(bist100.index, bist100['Adj Close'], c='r')
plt.title('BIST 100 Index')
plt.xlabel('')
plt.figtext(
    0.5,
    -0.02,
    'Data source: yfinance library. Index values are not adjusted after removing two zeros.',
    wrap=True,
    horizontalalignment='left',
    fontsize=10,
    style='italic',
    c='gray'
)
plt.show()

In [ ]:
def get_stock_data(symbol, start_date):

    df_yahoo = yf.download(symbol + '.IS', start=pd.to_datetime(start_date).strftime('%Y-%m-%d'))
    df_yahoo = df_yahoo[['Adj Close']].copy()
    df_yahoo.columns = ['Adj Close_yfinance']

    stock_data = StockData()
    df_isyatirimhisse = stock_data.get_data(
        symbols=symbol,
        start_date=start_date,
        exchange='0'
    )
    df_isyatirimhisse = df_isyatirimhisse[['DATE', 'CLOSING_TL']]
    df_isyatirimhisse.columns = ['Date', 'Adj Close_isyatirimhisse']

    df_yahoo['Date'] = pd.to_datetime(df_yahoo.index)
    df_isyatirimhisse['Date'] = pd.to_datetime(df_isyatirimhisse['Date'])

    df_yahoo = df_yahoo.set_index('Date')
    df_isyatirimhisse = df_isyatirimhisse.set_index('Date')

    merged_df = pd.merge(df_yahoo, df_isyatirimhisse, left_index=True, right_index=True)
    merged_df['Diff'] = merged_df['Adj Close_yfinance'] - merged_df['Adj Close_isyatirimhisse']

    merged_df.to_excel('check_prices.xlsx')

    return merged_df

def plot_stock_prices(merged_df):
    plt.figure(figsize=(10, 6))
    plt.plot(merged_df['Diff'])

    plt.title('Stock Prices Over Time Comparison')
    plt.xlabel('')
    plt.ylabel('Adjusted Close Price Difference (TL)')
    plt.show()

symbol = 'PGSUS'
start_date = '01-01-2010'

merged_df = get_stock_data(symbol, start_date)
plot_stock_prices(merged_df)